In [1]:
import gc
import glob

import geopandas as gpd
import momepy as mm

import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [2]:
regions_buildings_dir = '/data/uscuni-ulce/regions/buildings/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
simplfied_buildings_dir = '/data/uscuni-ulce/processed_data/simplified_buildings/'


regions_datadir = "/data/uscuni-ulce/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")

In [3]:
# df = pd.read_parquet('/data/uscuni-ulce/regions/cadastre_id_to_region.parquet')
# df.region.value_counts()
# '''
# region
# 4.0         11084489
# 107131.0     3078558
# 16242.0      1457482
# 4214.0       1389195
# 8707.0       1220411
#               ...   
# 7727.0         12682
# 6741.0         12235
# 10970.0        12170
# 10847.0        10871
# 15545.0        10409
# Name: count, Length: 828, dtype: int64



# df.region.value_counts().iloc[:20].index.values.astype(int)

# array([     4, 107131,  16242,   4214,   8707,  51289, 114311,  55763,
#         14679,  62954,  10579,  38679,  86873,  34266,     10,  80404,
#         18557,  69333,   1154, 130658])
# '''

##

In [4]:
# 107131 - krakow
# 86873 = vienna
# 69333 = prague
# munich
# 4 - rhineruhr
# 55763 - berlin
#bratislava - 99886
# freiburg - 5883
 # hamburg - 16242
# munich - 38679

In [9]:
region_name = 5883
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

In [10]:
# region_hulls.explore()

## Buildings


In [7]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [8]:
%%time

buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')

CPU times: user 52.3 ms, sys: 29 ms, total: 81.3 ms
Wall time: 77.9 ms


In [9]:
%%time

buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)

Percent polygons:  1.0



KeyboardInterrupt



In [10]:

# orig_buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
# dropped_buildings = orig_buildings[~orig_buildings['iid'].isin(buildings['iid'])]


# m = dropped_buildings.explore(color='red')
# m = buildings.explore(m=m)
# m

In [11]:
# buildings.to_parquet(simplfied_buildings_dir + f"buildings_{region_id}.parquet")

In [12]:
buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

KeyboardInterrupt: 

## Streets

In [13]:
from core.generate_streets import process_region_streets, process_single_region_streets

In [14]:
%%time
## processs streets
streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)

/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:343: UserWarning: An error occured at location POINT (4634734.833449237 3007874.1868347814). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads_cleaned = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:200: UserWarning: Could not create a connectio

CPU times: user 9min 10s, sys: 2.89 s, total: 9min 13s
Wall time: 9min 7s


In [15]:
# save streets
streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [16]:
# streets.explore()

In [17]:
# import geopandas as gpd
# gpd.read_parquet(streets_dir + f'streets_{region_id}.parquet').explore()

In [18]:
# gpd.read_parquet(enclosures_dir + f'enclosure_{region_id}.parquet').explore()

## Enclosures & Tessellation

In [14]:
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess

In [15]:
# 21894
# 47090 - problems with topolgical exceptions

In [16]:
### copy the original buildings
# !cp /data/uscuni-ulce/processed_data/simplified_buildings/buildings_69333_merged.parquet /data/uscuni-ulce/processed_data/buildings/buildings_69333.parquet

In [17]:
%%time
enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

---- Processing region:  55835 2024-11-17 12:32:16.043767
Retrying tesselation with less buildings, potentially changing building data.
Dropping 2 buildings due to tesselation problems
CPU times: user 34.4 s, sys: 795 ms, total: 35.2 s
Wall time: 48.3 s


In [18]:
enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
print("Processed enclosures")

## save files
tesselations.to_parquet(
    tessellations_dir + f"tessellation_{region_id}.parquet"
)
print("processed tesselations")

Processed enclosures
processed tesselations


In [19]:
# import lonboard
# layeasr = lonboard.SolidPolygonLayer.from_geopandas(tesselations, opacity=0.15)
# m = lonboard.Map([layer])
# m

In [20]:
# layer = lonboard.SolidPolygonLayer.from_geopandas(enclosures, opacity=0.15)
# m = lonboard.Map([layer])
# m

## Graphs

In [21]:
from core.generate_ngraphs import process_region_graphs

In [22]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1
Built nodes graph knn=1
CPU times: user 7.47 s, sys: 160 ms, total: 7.63 s
Wall time: 7.6 s


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 29 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


In [28]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]

In [29]:
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,


In [30]:
# from joblib import Parallel, delayed
# n_jobs = -1
# new = Parallel(n_jobs=n_jobs)(
#     delayed(process_region_graphs)(     region_id,
#                                         graph_dir,
#                                         buildings_dir,
#                                         streets_dir,
#                                         enclosures_dir,
#                                         tessellations_dir
#                                   ) 
#     for region_id, _ in region_hulls.iterrows())

In [31]:
1

1

## Characters

In [23]:
from core.generate_chars import process_single_region_chars

In [26]:
%%time
process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

2024-11-17 12:35:02.317576 ----Processing ------ 55835
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation
CPU times: user 1min 44s, sys: 871 ms, total: 1min 44s
Wall time: 1min 44s


In [24]:
from core.generate_chars import process_single_region_chars, process_street_chars, process_building_chars, process_tessellation_chars,process_enclosure_chars

In [25]:
# process_building_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [8]:
# %%time
# process_street_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [9]:
# process_tessellation_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [10]:
# process_enclosure_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

## Generate primary data

In [27]:
from core.generate_merged_primary_chars import merge_into_primary

In [30]:
%%time
merge_into_primary(region_id)

Processing region 47919
CPU times: user 1.24 s, sys: 612 ms, total: 1.86 s
Wall time: 1.54 s


## Generate morphotopes

In [11]:
from core.generate_clusters import process_single_region_morphotopes

In [12]:
process_single_region_morphotopes(region_id)

2024-11-22 16:34:20.049321 ----Processing ------ 5883
--------Generating lag----------
--------Generating morphotopes---------- 75
--------Dropping columns---------- ['stcSAl', 'stbOri', 'stcOri', 'stbCeA', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB', 'likWCe']
--------Generating morphotopes data----------
